In [1]:
from pyspark.sql import SparkSession

from pyspark.sql.types import StructType, IntegerType , StringType
from pyspark.sql.functions import *

In [2]:
spark=SparkSession.builder.appName('Streamed JSON records to CSV files').getOrCreate()

Picked up _JAVA_OPTIONS: -Dawt.useSystemAAFontSettings=on -Dswing.aatext=true
Picked up _JAVA_OPTIONS: -Dawt.useSystemAAFontSettings=on -Dswing.aatext=true
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/18 09:18:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
schema=StructType()\
    .add("id",IntegerType())\
    .add("name",StringType())\
    .add("host_id", IntegerType())\
    .add("neighbourhood_cleansed",StringType())\
    .add("price",StringType())

In [4]:
lines=spark\
        .readStream\
        .format('socket')\
        .option('host', 'localhost')\
        .option('port', 9999)\
        .load()\
        .select(from_json(col("value").cast("string"),schema).alias("json"))\
        .select(col("json.*"))

23/02/18 09:18:53 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [5]:
transformation=lines.withColumn('price', regexp_replace('price','([$])',''))\
                    .withColumn("price" , col("price").cast(IntegerType()))

In [6]:
query=transformation\
        .writeStream\
        .outputMode("append")\
        .format("csv")\
        .option("checkpointLocation", "checkpoint")\
        .option("path","output")\
        .start()

23/02/18 09:19:35 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


23/02/18 09:24:04 WARN TextSocketMicroBatchStream: Stream closed by localhost:9999


In [74]:
transformation.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- neighbourhood_cleansed: string (nullable = true)
 |-- price: integer (nullable = true)



In [75]:
query.stop()